# Fish detection application

In [1]:
# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Get video object
vidcap = cv2.VideoCapture('../../data/pladijs_plexiglas_licht.mp4')

In [3]:
# Read first image
success,image1 = vidcap.read()

# Loop
while True:
    
    # Load image
    success,image2 = vidcap.read()
    if not success:
        continue

    # Convert to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Smooth image
    smooth1 = cv2.GaussianBlur(gray1, ksize=(5,5), sigmaX=0)
    smooth2 = cv2.GaussianBlur(gray2, ksize=(5,5), sigmaX=0)

    # Calculate difference
    diff_frame = cv2.absdiff(smooth1, smooth2)

    # Only take different areas that are different enough
    thresh_frame = cv2.threshold(diff_frame, thresh=30, maxval=255, type=cv2.THRESH_BINARY)[1]

    # Closing
    kernel = np.ones((100, 100), np.uint8)
    closed_frame = cv2.morphologyEx(thresh_frame, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(closed_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours
    image_with_contours = cv2.drawContours(np.copy(image2), contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

    # Filter contours and draw bounding box
    image_with_box = np.copy(image2)
    for contour in contours:
        if cv2.contourArea(contour) > 2000:
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(image_with_box, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)

    # Display the image
    cv2.imshow('live video', image_with_box)
 
    # wait for user to press any key
    key = cv2.waitKey(1)

    # Renew image
    image1 = image2

KeyboardInterrupt: 